In [2]:
import keras
import json
import os.path
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop
import os
from os import environ

from keras.callbacks import TensorBoard
from keras.callbacks import LambdaCallback


Using TensorFlow backend.


In [3]:
batch_size = 128
num_classes = 10
epochs = 10
from sklearn.model_selection import train_test_split
import pickle

(X,y), (X_test,y_test) = mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=.166, random_state=42)
with open('mnist-nnm-train.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-nnm-valid.pkl', 'wb') as f:
    pickle.dump((X_valid, y_valid), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-nnm-test.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

11493376/11490434 [==============================] - 2s 0us/step


In [4]:
with open('mnist-nnm-train.pkl', 'rb') as f:
    (train_data, train_label) = pickle.load(f)
    if (len(train_data.shape) == 3): 
        if('tensorflow' == 'tensorflow'):
            train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], 1).astype('float32') / 255   
        else:
            train_data = train_data.reshape(train_data.shape[0], 1, train_data.shape[1], train_data.shape[2]).astype('float32') / 255   
    if (len(train_label.shape) == 1) or (len(train_label.shape) == 2 and train_label.shape[1] == 1):
        from keras.utils import np_utils
        class_labels_count = len(set(train_label.flatten()))
        train_label = np_utils.to_categorical(train_label, class_labels_count)
    else:
        class_labels_count = train_label.shape[1]

In [5]:
val_data = []
if('mnist-nnm-valid.pkl'):
    with open('mnist-nnm-valid.pkl', 'rb') as f:
        (val_data, val_label) = pickle.load(f)
        if (len(val_data.shape) == 3):
            if('tensorflow' == 'tensorflow'):
                val_data = val_data.reshape(val_data.shape[0], val_data.shape[1], val_data.shape[2], 1).astype('float32') / 255
            else:
                val_data = val_data.reshape(val_data.shape[0], 1, val_data.shape[1], val_data.shape[2]).astype('float32') / 255
        if (len(val_label.shape) == 1) or (len(val_label.shape) == 2 and val_label.shape[1] == 1):
            from keras.utils import np_utils
            val_label = np_utils.to_categorical(val_label, class_labels_count)

In [6]:
test_data = []
if('mnist-nnm-test.pkl'):
    with open('mnist-nnm-test.pkl', 'rb') as f:
        (test_data, test_label) = pickle.load(f)
        if (len(test_data.shape) == 3):
            if('tensorflow' == 'tensorflow'):
                test_data = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], 1).astype('float32') / 255
            else:
                test_data = test_data.reshape(test_data.shape[0], 1, test_data.shape[1], test_data.shape[2]).astype('float32') / 255
        if (len(test_label.shape) == 1) or (len(test_label.shape) == 2 and test_label.shape[1] == 1):
            from keras.utils import np_utils
            test_label = np_utils.to_categorical(test_label, class_labels_count)

In [7]:
train_data = train_data[0:1000]
train_label = train_label[0:1000]

val_data = val_data[0:100]
val_label = val_label[0:100]

test_data = test_data[0:100]
test_label = test_label[0:100]

In [49]:
model = Sequential()
model.add(Conv2D(4, kernel_size=(1, 1), strides=(1, 1), activation='relu', input_shape=train_data.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 4)         8         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 4)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                7850      
Total params: 7,858
Trainable params: 7,858
Non-trainable params: 0
_________________________________________________________________


In [66]:
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print([model.layers[0].get_weights()[1][1],model.layers[0].get_weights()[1][3]], '\n\n'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=.001),
              metrics=['accuracy'])



history = model.fit(train_data, train_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(val_data, val_label),
                    callbacks = [print_weights])



    #print("Training history:" + str(history.history))

score = model.evaluate(test_data, test_label, verbose=0)
print('Learning rate: ', rate)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print()


Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 1s 857us/step - loss: 1.9752e-06 - acc: 1.0000 - val_loss: 1.9390 - val_acc: 0.8500
[-0.082618676, -0.070572853] 


Epoch 2/10
1000/1000 [==============================] - 0s 48us/step - loss: 1.4586e-06 - acc: 1.0000 - val_loss: 1.9214 - val_acc: 0.8500
[-0.078564182, -0.066164061] 


Epoch 3/10
1000/1000 [==============================] - 0s 51us/step - loss: 1.0538e-06 - acc: 1.0000 - val_loss: 1.9262 - val_acc: 0.8500
[-0.075574763, -0.062933728] 


Epoch 4/10
1000/1000 [==============================] - 0s 69us/step - loss: 1.0551e-06 - acc: 1.0000 - val_loss: 1.9295 - val_acc: 0.8500
[-0.072895609, -0.059915192] 


Epoch 5/10
1000/1000 [==============================] - 0s 47us/step - loss: 9.4444e-07 - acc: 1.0000 - val_loss: 1.8971 - val_acc: 0.8500
[-0.069015302, -0.055998918] 


Epoch 6/10
1000/1000 [==============================] - 0s 71us/step - loss: 8.6803e-07 - acc: 1.

In [63]:
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print([model.layers[0].get_weights()[1][1],model.layers[0].get_weights()[1][3]], '\n\n'))

learningrates = [random.uniform(.001,.01) for l in range(10)]
for rate in learningrates:
    model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=rate),
              metrics=['accuracy'])



    history = model.fit(train_data, train_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(val_data, val_label),
                    verbose=0)



    #print("Training history:" + str(history.history))

    score = model.evaluate(test_data, test_label, verbose=0)
    print('Learning rate: ', rate)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print()




Learning rate:  0.006223363349255782
Test loss: 1.0274930264
Test accuracy: 0.89

Learning rate:  0.00922439176771949
Test loss: 1.11073339033
Test accuracy: 0.89

Learning rate:  0.004136950877084511
Test loss: 1.08300776355
Test accuracy: 0.89

Learning rate:  0.002969408467028411
Test loss: 1.16545931348
Test accuracy: 0.89

Learning rate:  0.00828743255947171
Test loss: 1.00033042821
Test accuracy: 0.9

Learning rate:  0.009113602478041181
Test loss: 0.999704417568
Test accuracy: 0.9

Learning rate:  0.009918517143838065
Test loss: 1.00830812201
Test accuracy: 0.89

Learning rate:  0.0025412695851533833
Test loss: 1.11020659208
Test accuracy: 0.89

Learning rate:  0.009191889747518826
Test loss: 1.11185001254
Test accuracy: 0.88

Learning rate:  0.00470616138972089
Test loss: 1.11637908459
Test accuracy: 0.88

